In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
from itertools import combinations
import random
import glob

from keras.models import Model, Input, Sequential, load_model
from keras.applications.vgg16 import VGG16
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D, ZeroPadding2D, Dropout
from keras.optimizers import SGD,Adam

from src.data import process_audio
from src.data import display_audio
from src.models import one_shot_learning


In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Save model

In [ ]:
siamese_model_1.save('/Users/greenapple/project5/models/siamese_model_1.h5') # save model to disc

In [ ]:
siamese_model_2.save('/Users/greenapple/project5/models/02_siamese_model.h5') # save model to disc

In [ ]:
# Training on 100 speakers 1 epoch
siamese_model_2.save('/Users/greenapple/project5/models/02_2_siamese_model.h5') # save model to disc

In [ ]:
VGG16_like.save('/Users/greenapple/project5/models/VGG16_like.h5') # save model to disc

In [ ]:
siamese_model_1.save_weights('/Users/greenapple/project5/models/01_siamese_weights.h5')

 # Load model

In [ ]:
siamese_model_1 = load_model('/Users/greenapple/project5/models/01_siamese_model.h5') # load model

In [ ]:
siamese_model_1.summary() # model summary

In [ ]:
score = siamese_model_1.evaluate(X, y, verbose=0) # evaluate model

# Data sets

In [5]:
# Number of speakers in the train set
path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
id_folder_list =  glob.glob(os.path.join(path, '*'))
len(id_folder_list)

921

In [6]:
# Number of speakers in the test set
path = '/Users/greenapple/project5/data/raw/LibriSpeech/dev-clean'
id_folder_list =  glob.glob(os.path.join(path, '*'))
len(id_folder_list)

40

In [7]:
# Number of speakers in the dev set
path = '/Users/greenapple/project5/data/raw/LibriSpeech/test-clean'
id_folder_list =  glob.glob(os.path.join(path, '*'))
len(id_folder_list)

40

# N-way One Shot learning

In [28]:
# Siamese model with VGG16 net as branches trained on 10 speakers, 10 epocks
# and one epoch on 100
siamese_model_2_2 = load_model('/Users/greenapple/project5/models/02_2_siamese_model.h5') # load model

# Parameters for one shot learning
model = siamese_model_2_2
trials = 100
n_way = 2

train_path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
test_path = '/Users/greenapple/project5/data/raw/LibriSpeech/test-clean'

speaker_num_tr = 10
speaker_num_ts = 10

speaker_data_train = one_shot_learning.get_samples_in_order(train_path, speaker_num_tr)
speaker_data_test = one_shot_learning.get_samples_random(test_path, speaker_num_ts)

one_shot_5_train = one_shot_learning.one_shot_score(model, trials, speaker_data_train, n_way)
one_shot_5_test = one_shot_learning.one_shot_score(model, trials, speaker_data_test, n_way)

print('2-way one shot train score is {}, test score is {}'.format(one_shot_5_train, one_shot_5_test))

2-way one shot train score is 92.0, test score is 63.0


# Compare models

In [34]:
# Test set: Siamese model with VGG16 net as branches trained on 10 speakers 

classes = [1, 2, 5]

# Siamese model with VGG16 net as branches trained on 100 speakers
siamese_model_2_2 = load_model('/Users/greenapple/project5/models/02_2_siamese_model.h5') # load model

model = siamese_model_2_2
trials = 20
test_path = '/Users/greenapple/project5/data/raw/LibriSpeech/test-clean'
speaker_num = 10

speaker_data_test = one_shot_learning.get_samples_random(test_path, speaker_num_ts)

test_one_shot_test_l_siamese2_2 = []

for n_way in classes:
    one_shot_test = one_shot_learning.one_shot_score(model, trials, speaker_data_test, n_way)
    test_one_shot_test_l_siamese2_2.append(one_shot_test)
    
print(test_one_shot_test_l_siamese2_2)


[100.0, 80.0, 65.0]core 65.057894736842114


In [37]:
test_one_shot_test_l_siamese2_2

[100.0, 80.0, 65.0]

In [38]:
# Train set: Siamese model with VGG16 net as branches trained on 10 speakers 

classes = [1, 2, 5]

# Siamese model with VGG16 net as branches trained on 100 speakers
siamese_model_2_2 = load_model('/Users/greenapple/project5/models/02_2_siamese_model.h5') # load model

model = siamese_model_2_2
trials = 20
train_path = '/Users/greenapple/project5/data/raw/LibriSpeech/train-clean-360'
speaker_num = 10

speaker_data_train = one_shot_learning.get_samples_in_order(train_path, speaker_num_tr)

test_one_shot_train_l_siamese2_2 = []

for n_way in classes:
    one_shot_test = one_shot_learning.one_shot_score(model, trials, speaker_data_train, n_way)
    test_one_shot_train_l_siamese2_2.append(one_shot_test)
    
print(test_one_shot_train_l_siamese2_2)


[100.0, 85.0, 65.0]core 65.057894736842176


In [36]:
test_one_shot_train_l_siamese2_2

[100.0, 80.0, 65.0]

In [40]:
# Random guess
one_shot_test_l_random = [100, 50, 20]

In [41]:
one_shot_test_l_random

[100, 50, 20]